In [1]:
## Import packages

#Kspice
import sys
sys.path.append(r"C:\Program Files (x86)\Kongsberg\K-Spice\bin64") #add to path to allow kspice import
import kspice # if import error, check correct python version (3.11)

#Basic functionality
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#ML
import torch 

In [2]:
#Initialize simulator

project_path = r"C:\Appl\K-Spice-Projects\Kristin_v23" #Specify path to downloaded project.
_ = kspice.Simulator(project_path) #Create instance of project

In [3]:
#Disable power and wells - not API functionality

In [4]:
timeline = _.activate_timeline("Engineering") # Select the avaliable engineering timeline
app = "Topside" # We only make changes to the topside module NOTE: From software we can #deactivate Wells and Power in ESS model, can this be done from python? (If it increases speed)
timeline.initialize() #

In [5]:
#Load Kristin model, parameters and initial conditions

timeline.load_model("KristinMaria_master_disabledPowerWells") #Load model
timeline.load_parameters("KristinMaria_master_disabledPowerWells") # load fixed parameters
timeline.load_initial_condition("KristinMaria_master_disabledPowerWells") # Load initial conditions

In [40]:
import sys
del sys.modules['enviroment'] #to delete package memory - loading updates of Sim class
from enviroment import Sim

env = Sim(timeline, app) # Create env instance
env.import_variables("xl_tester1.xlsx") #Import variables of interest

In [34]:
env.check_df("Global KPI") #Check random column in dataframe. i.e. correct labeling in correspondance to simulator

In [35]:
env.timeline.set_speed(10) #Set speed of simulator, this value is what we wish for not necesarly what we get
env.timeline.run()

RuntimeError: Run failed

In [36]:
env.timeline.achieved_speed #Here you can see actual speed

5.105172233316482

In [10]:
state = env.reset() # We reset the process simulator by pausing it and again fetching the initial conditions

In [11]:
def select_action(state):
    """ To ensure the model explores new spaces we will sometimes choose actions randomly. If not random we choose the action which result in the highest expected reward. 
    Choosing a random action will decay exponientially throughout learning. 
    """
    EPS_START = 0.9
    EPS_END = 0.05
    EPS_DECAY = 1000

    global steps_done
    sample = np.random.rand(1)
    eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor(env.sample("action"), dtype=torch.float32) # random action

In [12]:
#For the future we want the state at a specific form 
import itertools
state_flat = np.array(list(itertools.chain.from_iterable(state)))
state_tensor = torch.tensor(state_flat, dtype=torch.float32).unsqueeze(0) # make torch tensor and add axis

In [37]:
steps_done = 0
action = select_action(state_tensor) # select action (up, down, stay) for each controller by a factor of 1.
print(int(action[0].item()))

1


In [44]:
observation, reward, terminated, truncated = env.step(action) # Add changes to model and use a polling function to check for updates, timeout currently as 5 min to reach new setpoint with arg: 0.1 precision

S23FIT1014 has reached its setpoint 13.214115822351046 [m3/h]
[0, 1, 2, 3, 4, 5, 7, 8, 9]
S23TT1004 has reached its setpoint 29.798600280504388 [C]
[1, 2, 3, 4, 5, 7, 8, 9]
S23FIT1107 has reached its setpoint 30.09363135686255 [m3/h]
[1, 2, 3, 4, 5, 7, 9]
S23TT1034 has reached its setpoint 30.200608697008477 [C]
[1, 3, 4, 5, 7, 9]
S23LT1036 has reached its setpoint 45.20086962023436 [%]
[1, 4, 5, 7, 9]
S23LT1006 has reached its setpoint 44.79613583610609 [%]
[4, 5, 7, 9]
S23LT1066 has reached its setpoint 41.799845120437 [%]
[4, 7, 9]
Timeout reached


In [17]:
env.timeline.pause() # Pause simulator